In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('myproj').getOrCreate()
data = spark.read.csv('/FileStore/tables/UCI_Credit_Card.csv',inferSchema=True,header=True)
data.printSchema()

root
-- ID: integer (nullable = true)
-- LIMIT_BAL: double (nullable = true)
-- SEX: integer (nullable = true)
-- EDUCATION: integer (nullable = true)
-- MARRIAGE: integer (nullable = true)
-- AGE: integer (nullable = true)
-- PAY_0: integer (nullable = true)
-- PAY_2: integer (nullable = true)
-- PAY_3: integer (nullable = true)
-- PAY_4: integer (nullable = true)
-- PAY_5: integer (nullable = true)
-- PAY_6: integer (nullable = true)
-- BILL_AMT1: double (nullable = true)
-- BILL_AMT2: double (nullable = true)
-- BILL_AMT3: double (nullable = true)
-- BILL_AMT4: double (nullable = true)
-- BILL_AMT5: double (nullable = true)
-- BILL_AMT6: double (nullable = true)
-- PAY_AMT1: double (nullable = true)
-- PAY_AMT2: double (nullable = true)
-- PAY_AMT3: double (nullable = true)
-- PAY_AMT4: double (nullable = true)
-- PAY_AMT5: double (nullable = true)
-- PAY_AMT6: double (nullable = true)
-- default.payment.next.month: integer (nullable = true)

Data Dictionary
1.	ID: ID of each client
2.	LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit)
3.	SEX: Gender (1=male, 2=female)
4.	EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
5.	MARRIAGE: Marital status (1=married, 2=single, 3=others)
6.	Age: Age in years
7.	PAY_0 ------ PAY_6 (6 features): Repayment status from April, 2005 to September, 2005(-1=pay duly, 0=not delay, other number = the number of months for payment delay)
8.	BILL_AMT1 ------ BILL_AMT6 (6 features): Amount of bill statement from April, 2005 to September 2005(NT dollar)
9.	PAY_AMT1: Amount of previous payment from April, 2005 to September 2005(NT dollar)
10.	Default. Payment. next. Month: default payment (1=yes, 2=no)

In [0]:
data = data.withColumnRenamed("PAY_0", "PAY_1")
data = data.withColumnRenamed("default.payment.next.month", "Default")

Change data types

In [0]:
from pyspark.sql.types import StructField,StringType,IntegerType,DoubleType,StructType
# if the second element is True means it could have null cells
df_schema = StructType([
    StructField('ID', StringType(), True),
    StructField('LIMIT_BAL', DoubleType(), True),
    StructField('SEX', StringType(), True),
    StructField('EDUCATION', StringType(), True),
    StructField('MARRIAGE', StringType(), True),
    StructField('AGE', IntegerType(), True),
    StructField('PAY_1', IntegerType(), True),
    StructField('PAY_2', IntegerType(), True),
    StructField('PAY_3', IntegerType(), True),
    StructField('PAY_4', IntegerType(), True),
    StructField('PAY_5', IntegerType(), True),
    StructField('PAY_6', IntegerType(), True),
    StructField('BILL_AMT1', DoubleType(), True),
    StructField('BILL_AMT2', DoubleType(), True),
    StructField('BILL_AMT3', DoubleType(), True),
    StructField('BILL_AMT4', DoubleType(), True),
    StructField('BILL_AMT5', DoubleType(), True),
    StructField('BILL_AMT6', DoubleType(), True),
    StructField('PAY_AMT1', DoubleType(), True),
    StructField('PAY_AMT2', DoubleType(), True),
    StructField('PAY_AMT3', DoubleType(), True),
    StructField('PAY_AMT4', DoubleType(), True),
    StructField('PAY_AMT5', DoubleType(), True),
    StructField('PAY_AMT6', DoubleType(), True),
    StructField('Default', IntegerType(), True),
    ])
data = spark.createDataFrame(data.collect(),schema = df_schema)
data.printSchema()

root
-- ID: string (nullable = true)
-- LIMIT_BAL: double (nullable = true)
-- SEX: string (nullable = true)
-- EDUCATION: string (nullable = true)
-- MARRIAGE: string (nullable = true)
-- AGE: integer (nullable = true)
-- PAY_1: integer (nullable = true)
-- PAY_2: integer (nullable = true)
-- PAY_3: integer (nullable = true)
-- PAY_4: integer (nullable = true)
-- PAY_5: integer (nullable = true)
-- PAY_6: integer (nullable = true)
-- BILL_AMT1: double (nullable = true)
-- BILL_AMT2: double (nullable = true)
-- BILL_AMT3: double (nullable = true)
-- BILL_AMT4: double (nullable = true)
-- BILL_AMT5: double (nullable = true)
-- BILL_AMT6: double (nullable = true)
-- PAY_AMT1: double (nullable = true)
-- PAY_AMT2: double (nullable = true)
-- PAY_AMT3: double (nullable = true)
-- PAY_AMT4: double (nullable = true)
-- PAY_AMT5: double (nullable = true)
-- PAY_AMT6: double (nullable = true)
-- Default: integer (nullable = true)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import when, count, col
df = data
na_report=df.select([count(when(isnull(c), c)).alias(c) for c in df.columns])# dimensions of the dataframe
print("Number of Rows: ",df.count() ,"   Number of Columns: ", len(df.columns))
# we should handle the missing values by imputation unless too many of them are emtpy
# na.drop() method is not recommended for avoiding biased except for the dependent variable
na_report.show()

Number of Rows: 30000 Number of Columns: 25
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
 ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_1|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|Default|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+

In [0]:
df.describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+----------------+------------------+--------------------+------------------+--------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+
summary| ID| LIMIT_BAL| SEX| EDUCATION| MARRIAGE| AGE| PAY_1| PAY_2| PAY_3| PAY_4| PAY_5| PAY_6| BILL_AMT1| BILL_AMT2| BILL_AMT3| BILL_AMT4| BILL_AMT5| BILL_AMT6| PAY_AMT1| PAY_AMT2| PAY_AMT3| PAY_AMT4| PAY_AMT5| PAY_AMT6| Default|
+-------+-----------------+------------------+------------------+------------------+------------------+----------------+------------------+--------------------+------------------+--------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+
 count| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000|
 mean| 15000.5|167484.32266666667|1.6037333333333332|1.8531333333333333|1.5518666666666667| 35.4855| -0.0167|-0.13376666666666667| -0.1662|-0.22066666666666668| -0.2662| -0.2911| 51223.3309|49179.07516666667| 47013.1548| 43262.94896666666|40311.40096666667| 38871.7604| 5663.5805| 5921.1635| 5225.6815|4826.076866666666| 4799.387633333334| 5215.502566666667| 0.2212|
 stddev|8660.398374208891|129747.66156720239|0.4891291960902606|0.7903486597207294|0.5219696006132485|9.21790406809019|1.1238015279973346| 1.1971859730345529|1.1968675684465735| 1.1691386224023377|1.1331874060027485|1.1499876256079022|73635.86057552954|71173.76878252836|69349.38742703685|64332.856133916335|60797.15577026485|59554.10753674574|16563.280354025774|23040.87040205723|17606.961469803104| 15666.159744032|15278.305679144783|17777.465775435303|0.4150618056909332|
 min| 1| 10000.0| 1| 0| 0| 21| -2| -2| -2| -2| -2| -2| -165580.0| -69777.0| -157264.0| -170000.0| -81334.0| -339603.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0|
 max| 9999| 1000000.0| 2| 6| 3| 79| 8| 8| 8| 8| 8| 8| 964511.0| 983931.0| 1664089.0| 891586.0| 927171.0| 961664.0| 873552.0| 1684259.0| 896040.0| 621000.0| 426529.0| 528666.0| 1|
+-------+-----------------+------------------+------------------+------------------+------------------+----------------+------------------+--------------------+------------------+--------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+

Drop abnormal data in "EDUCATION" and "MARRIAGE" comlumns

In [0]:
df.select('MARRIAGE','EDUCATION').describe().show()

+-------+------------------+------------------+
summary| MARRIAGE| EDUCATION|
+-------+------------------+------------------+
 count| 30000| 30000|
 mean|1.5518666666666667|1.8531333333333333|
 stddev|0.5219696006132485|0.7903486597207294|
 min| 0| 0|
 max| 3| 6|
+-------+------------------+------------------+

In [0]:
df = df.filter((df.MARRIAGE != '3')&(df.MARRIAGE != '0') &(df.EDUCATION != '0') &  (df.EDUCATION != '5') & (df.EDUCATION != '6'))

In [0]:
df.select('MARRIAGE','EDUCATION').describe().show()

+-------+------------------+------------------+
summary| MARRIAGE| EDUCATION|
+-------+------------------+------------------+
 count| 29283| 29283|
 mean|1.5397671003653997| 1.8114605743947|
 stddev|0.4984245794401454|0.7094951994131745|
 min| 1| 1|
 max| 2| 4|
+-------+------------------+------------------+

Visualizations

In [0]:
df.select("EDUCATION","Default","ID").groupBy("EDUCATION","Default").agg(count("ID")).orderBy('EDUCATION').display()

EDUCATION,Default,count(ID)
1,0,8508
1,1,2023
2,0,10577
2,1,3285
3,1,1206
3,0,3564
4,0,113
4,1,7


In [0]:
df.select("SEX","Default","ID").groupBy("SEX","Default").agg(count("ID")).display()

SEX,Default,count(ID)
1,0,8795
1,1,2821
2,0,13967
2,1,3700


In [0]:
df.select("AGE","Default","ID").groupBy("AGE","Default").agg(count("ID")).orderBy('AGE').display()

AGE,Default,count(ID)
21,1,14
21,0,50
22,0,383
22,1,164
23,1,246
23,0,669
24,0,815
24,1,300
25,1,298
25,0,873


In [0]:
df.select("EDUCATION","ID").groupBy("EDUCATION").agg(count("ID")).orderBy('EDUCATION').display()

EDUCATION,count(ID)
1,10531
2,13862
3,4770
4,120


In [0]:
df.select("MARRIAGE","ID").groupBy("MARRIAGE").agg(count("ID")).display()

MARRIAGE,count(ID)
1,13477
2,15806


In [0]:
df.select("MARRIAGE","ID","Default").groupBy("Default","MARRIAGE").agg(count("id")).orderBy('MARRIAGE').display()

Default,MARRIAGE,count(id)
0,1,10285
1,1,3192
0,2,12477
1,2,3329


In [0]:
df.select("LIMIT_BAL","ID","Default").groupBy("Default","LIMIT_BAL").agg(count("id")).orderBy('LIMIT_BAL').display()

Default,LIMIT_BAL,count(id)
1,10000.0,189
0,10000.0,284
0,16000.0,1
1,20000.0,679
0,20000.0,1231
0,30000.0,1003
1,30000.0,554
1,40000.0,92
0,40000.0,132
0,50000.0,2397


In [0]:
df.select("Default","ID").groupBy("Default").agg(count("ID")).display()

Default,count(ID)
1,6521
0,22762


Check outliers

In [0]:
quantiles = {
    c: dict(
        zip(["q1", "q3"], df.approxQuantile(c, [0.25, 0.75], 0))
    )
    for c in ["LIMIT_BAL", "AGE","PAY_1","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6","BILL_AMT1","BILL_AMT2","BILL_AMT3","BILL_AMT4","BILL_AMT5","BILL_AMT6","PAY_AMT1","PAY_AMT2","PAY_AMT3","PAY_AMT4","PAY_AMT5","PAY_AMT6"]
}
quantiles

Out[17]: {'LIMIT_BAL': {'q1': 50000.0, 'q3': 240000.0},
 'AGE': {'q1': 28.0, 'q3': 41.0},
 'PAY_1': {'q1': -1.0, 'q3': 0.0},
 'PAY_2': {'q1': -1.0, 'q3': 0.0},
 'PAY_3': {'q1': -1.0, 'q3': 0.0},
 'PAY_4': {'q1': -1.0, 'q3': 0.0},
 'PAY_5': {'q1': -1.0, 'q3': 0.0},
 'PAY_6': {'q1': -1.0, 'q3': 0.0},
 'BILL_AMT1': {'q1': 3491.0, 'q3': 66866.0},
 'BILL_AMT2': {'q1': 2946.0, 'q3': 63774.0},
 'BILL_AMT3': {'q1': 2618.0, 'q3': 60005.0},
 'BILL_AMT4': {'q1': 2313.0, 'q3': 54574.0},
 'BILL_AMT5': {'q1': 1770.0, 'q3': 50290.0},
 'BILL_AMT6': {'q1': 1271.0, 'q3': 49327.0},
 'PAY_AMT1': {'q1': 995.0, 'q3': 5007.0},
 'PAY_AMT2': {'q1': 817.0, 'q3': 5000.0},
 'PAY_AMT3': {'q1': 390.0, 'q3': 4508.0},
 'PAY_AMT4': {'q1': 292.0, 'q3': 4038.0},
 'PAY_AMT5': {'q1': 260.0, 'q3': 4086.0},
 'PAY_AMT6': {'q1': 134.0, 'q3': 4003.0}}

In [0]:
for i in quantiles:
    iqr = quantiles[i]['q3'] - quantiles[i]['q1']
    quantiles[i]['lower_bound'] = quantiles[i]['q1'] - (iqr * 1.5)
    quantiles[i]['upper_bound'] = quantiles[i]['q3'] + (iqr * 1.5)
print(quantiles)

{'LIMIT_BAL': {'q1': 50000.0, 'q3': 240000.0, 'lower_bound': -235000.0, 'upper_bound': 525000.0}, 'AGE': {'q1': 28.0, 'q3': 41.0, 'lower_bound': 8.5, 'upper_bound': 60.5}, 'PAY_1': {'q1': -1.0, 'q3': 0.0, 'lower_bound': -2.5, 'upper_bound': 1.5}, 'PAY_2': {'q1': -1.0, 'q3': 0.0, 'lower_bound': -2.5, 'upper_bound': 1.5}, 'PAY_3': {'q1': -1.0, 'q3': 0.0, 'lower_bound': -2.5, 'upper_bound': 1.5}, 'PAY_4': {'q1': -1.0, 'q3': 0.0, 'lower_bound': -2.5, 'upper_bound': 1.5}, 'PAY_5': {'q1': -1.0, 'q3': 0.0, 'lower_bound': -2.5, 'upper_bound': 1.5}, 'PAY_6': {'q1': -1.0, 'q3': 0.0, 'lower_bound': -2.5, 'upper_bound': 1.5}, 'BILL_AMT1': {'q1': 3491.0, 'q3': 66866.0, 'lower_bound': -91571.5, 'upper_bound': 161928.5}, 'BILL_AMT2': {'q1': 2946.0, 'q3': 63774.0, 'lower_bound': -88296.0, 'upper_bound': 155016.0}, 'BILL_AMT3': {'q1': 2618.0, 'q3': 60005.0, 'lower_bound': -83462.5, 'upper_bound': 146085.5}, 'BILL_AMT4': {'q1': 2313.0, 'q3': 54574.0, 'lower_bound': -76078.5, 'upper_bound': 132965.5}, 'BILL_AMT5': {'q1': 1770.0, 'q3': 50290.0, 'lower_bound': -71010.0, 'upper_bound': 123070.0}, 'BILL_AMT6': {'q1': 1271.0, 'q3': 49327.0, 'lower_bound': -70813.0, 'upper_bound': 121411.0}, 'PAY_AMT1': {'q1': 995.0, 'q3': 5007.0, 'lower_bound': -5023.0, 'upper_bound': 11025.0}, 'PAY_AMT2': {'q1': 817.0, 'q3': 5000.0, 'lower_bound': -5457.5, 'upper_bound': 11274.5}, 'PAY_AMT3': {'q1': 390.0, 'q3': 4508.0, 'lower_bound': -5787.0, 'upper_bound': 10685.0}, 'PAY_AMT4': {'q1': 292.0, 'q3': 4038.0, 'lower_bound': -5327.0, 'upper_bound': 9657.0}, 'PAY_AMT5': {'q1': 260.0, 'q3': 4086.0, 'lower_bound': -5479.0, 'upper_bound': 9825.0}, 'PAY_AMT6': {'q1': 134.0, 'q3': 4003.0, 'lower_bound': -5669.5, 'upper_bound': 9806.5}}

In [0]:

import pyspark.sql.functions as f
df_clean=df.select(
    "*",
    *[
        f.when(
            f.col(c).between(quantiles[c]['lower_bound'], quantiles[c]['upper_bound']),
            0
        ).otherwise(1).alias(c+"_out") 
        for c in ["LIMIT_BAL", "AGE","PAY_1","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6","BILL_AMT1","BILL_AMT2","BILL_AMT3","BILL_AMT4","BILL_AMT5","BILL_AMT6","PAY_AMT1","PAY_AMT2","PAY_AMT3","PAY_AMT4","PAY_AMT5","PAY_AMT6"]
    ]
)

In [0]:
from pyspark.sql.functions import col
df_clean=df_clean.withColumn("outliers", col("LIMIT_BAL_out")+col("AGE_out")+col("PAY_1_out")+col("BILL_AMT1_out")+col("PAY_AMT1_out")+col("PAY_2_out")+col("BILL_AMT2_out")+col("PAY_AMT2_out")+col("PAY_3_out")+col("BILL_AMT3_out")+col("PAY_AMT3_out")+col("PAY_4_out")+col("BILL_AMT4_out")+col("PAY_AMT4_out")+col("PAY_5_out")+col("BILL_AMT5_out")+col("PAY_AMT5_out")+col("PAY_6_out")+col("BILL_AMT6_out")+col("PAY_AMT6_out"))

In [0]:
df_clean.select("outliers","ID").groupBy("outliers").agg(count("id")).orderBy('outliers').display()

outliers,count(id)
0,12827
1,5797
2,3040
3,2001
4,1397
5,1044
6,1635
7,454
8,259
9,188


Data processing

Dealing with imbalanced labels（Try to improve the performance---Oversampling）

In [0]:
majority = df.filter((df.Default == 0))
minority = df.filter((df.Default == 1))
 
majority_count = majority.count()
minority_count = minority.count() 
ratio = majority_count / minority_count
print("ratio: {}".format(ratio))
sampled_minority_df = minority.sample(withReplacement=True, fraction=ratio)
 
df = sampled_minority_df.unionAll(majority)
df.show()

ratio: 3.4905689311455297
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
 ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_1|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|Default|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
 1| 20000.0| 2| 2| 1| 24| 2| 2| -1| -1| -2| -2| 3913.0| 3102.0| 689.0| 0.0| 0.0| 0.0| 0.0| 689.0| 0.0| 0.0| 0.0| 0.0| 1|
 1| 20000.0| 2| 2| 1| 24| 2| 2| -1| -1| -2| -2| 3913.0| 3102.0| 689.0| 0.0| 0.0| 0.0| 0.0| 689.0| 0.0| 0.0| 0.0| 0.0| 1|
 1| 20000.0| 2| 2| 1| 24| 2| 2| -1| -1| -2| -2| 3913.0| 3102.0| 689.0| 0.0| 0.0| 0.0| 0.0| 689.0| 0.0| 0.0| 0.0| 0.0| 1|
 1| 20000.0| 2| 2| 1| 24| 2| 2| -1| -1| -2| -2| 3913.0| 3102.0| 689.0| 0.0| 0.0| 0.0| 0.0| 689.0| 0.0| 0.0| 0.0| 0.0| 1|
 1| 20000.0| 2| 2| 1| 24| 2| 2| -1| -1| -2| -2| 3913.0| 3102.0| 689.0| 0.0| 0.0| 0.0| 0.0| 689.0| 0.0| 0.0| 0.0| 0.0| 1|
 1| 20000.0| 2| 2| 1| 24| 2| 2| -1| -1| -2| -2| 3913.0| 3102.0| 689.0| 0.0| 0.0| 0.0| 0.0| 689.0| 0.0| 0.0| 0.0| 0.0| 1|
 1| 20000.0| 2| 2| 1| 24| 2| 2| -1| -1| -2| -2| 3913.0| 3102.0| 689.0| 0.0| 0.0| 0.0| 0.0| 689.0| 0.0| 0.0| 0.0| 0.0| 1|
 2| 120000.0| 2| 2| 2| 26| -1| 2| 0| 0| 0| 2| 2682.0| 1725.0| 2682.0| 3272.0| 3455.0| 3261.0| 0.0| 1000.0| 1000.0| 1000.0| 0.0| 2000.0| 1|
 2| 120000.0| 2| 2| 2| 26| -1| 2| 0| 0| 0| 2| 2682.0| 1725.0| 2682.0| 3272.0| 3455.0| 3261.0| 0.0| 1000.0| 1000.0| 1000.0| 0.0| 2000.0| 1|
 2| 120000.0| 2| 2| 2| 26| -1| 2| 0| 0| 0| 2| 2682.0| 1725.0| 2682.0| 3272.0| 3455.0| 3261.0| 0.0| 1000.0| 1000.0| 1000.0| 0.0| 2000.0| 1|
 2| 120000.0| 2| 2| 2| 26| -1| 2| 0| 0| 0| 2| 2682.0| 1725.0| 2682.0| 3272.0| 3455.0| 3261.0| 0.0| 1000.0| 1000.0| 1000.0| 0.0| 2000.0| 1|
 14| 70000.0| 1| 2| 2| 30| 1| 2| 2| 0| 0| 2| 65802.0| 67369.0| 65701.0| 66782.0| 36137.0| 36894.0| 3200.0| 0.0| 3000.0| 3000.0| 1500.0| 0.0| 1|
 14| 70000.0| 1| 2| 2| 30| 1| 2| 2| 0| 0| 2| 65802.0| 67369.0| 65701.0| 66782.0| 36137.0| 36894.0| 3200.0| 0.0| 3000.0| 3000.0| 1500.0| 0.0| 1|
 14| 70000.0| 1| 2| 2| 30| 1| 2| 2| 0| 0| 2| 65802.0| 67369.0| 65701.0| 66782.0| 36137.0| 36894.0| 3200.0| 0.0| 3000.0| 3000.0| 1500.0| 0.0| 1|
 14| 70000.0| 1| 2| 2| 30| 1| 2| 2| 0| 0| 2| 65802.0| 67369.0| 65701.0| 66782.0| 36137.0| 36894.0| 3200.0| 0.0| 3000.0| 3000.0| 1500.0| 0.0| 1|
 17| 20000.0| 1| 1| 2| 24| 0| 0| 2| 2| 2| 2| 15376.0| 18010.0| 17428.0| 18338.0| 17905.0| 19104.0| 3200.0| 0.0| 1500.0| 0.0| 1650.0| 0.0| 1|
 17| 20000.0| 1| 1| 2| 24| 0| 0| 2| 2| 2| 2| 15376.0| 18010.0| 17428.0| 18338.0| 17905.0| 19104.0| 3200.0| 0.0| 1500.0| 0.0| 1650.0| 0.0| 1|
 22| 120000.0| 2| 2| 1| 39| -1| -1| -1| -1| -1| -1| 316.0| 316.0| 316.0| 0.0| 632.0| 316.0| 316.0| 316.0| 0.0| 632.0| 316.0| 0.0| 1|
 22| 120000.0| 2| 2| 1| 39| -1| -1| -1| -1| -1| -1| 316.0| 316.0| 316.0| 0.0| 632.0| 316.0| 316.0| 316.0| 0.0| 632.0| 316.0| 0.0| 1|
 22| 120000.0| 2| 2| 1| 39| -1| -1| -1| -1| -1| -1| 316.0| 316.0| 316.0| 0.0| 632.0| 316.0| 316.0| 316.0| 0.0| 632.0| 316.0| 0.0| 1|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
only showing top 20 rows

In [0]:
df.select('Default').describe().show()

+-------+------------------+
summary| Default|
+-------+------------------+
 count| 45541|
 mean|0.5001866450012077|
 stddev|0.5000054548125263|
 min| 0|
 max| 1|
+-------+------------------+

In [0]:
df.select("Default","ID").groupBy("Default").agg(count("ID")).display()

Default,count(ID)
1,22779
0,22762


For statistical imputation we convert the pyspark dataframe into pandas to use functions of this library

In [0]:
from sklearn.impute import SimpleImputer
import pandas as pd

In [0]:
sample=df.toPandas()

In [0]:
samplecopy=sample
samplecopy

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Default
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
2,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
3,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
4,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45536,29991,140000.0,1,2,1,41,0,0,0,0,0,0,138325.0,137142.0,139110.0,138262.0,49675.0,46121.0,6000.0,7000.0,4228.0,1505.0,2000.0,2000.0,0
45537,29993,10000.0,1,3,1,43,0,0,0,-2,-2,-2,8802.0,10400.0,0.0,0.0,0.0,0.0,2000.0,0.0,0.0,0.0,0.0,0.0,0
45538,29994,100000.0,1,1,2,38,0,-1,-1,0,0,0,3042.0,1427.0,102996.0,70626.0,69473.0,55004.0,2000.0,111784.0,4000.0,3000.0,2000.0,2000.0,0
45539,29996,220000.0,1,3,1,39,0,0,0,0,0,0,188948.0,192815.0,208365.0,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0


Working with Categorical Columns

In [0]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [0]:
# OneHot encoding of categorical variable to be able to enter into machine learning algorithms
# we vectorize them to be able to use it
# Indexer return the numrical equivalent it's could be used for label encoding
# OneHotEncoder is for using returning dummy variables equivalent of the categorical variables
SEX_indexer = StringIndexer(inputCol='SEX',outputCol='SEXIndex')
SEX_encoder = OneHotEncoder(inputCol='SEXIndex',outputCol='SEXVec')

EDUCATION_indexer = StringIndexer(inputCol='EDUCATION',outputCol='EDUCATIONIndex')
EDUCATION_encoder = OneHotEncoder(inputCol='EDUCATIONIndex',outputCol='EDUCATIONVec')

MARRIAGE_indexer = StringIndexer(inputCol='MARRIAGE',outputCol='MARRIAGEIndex')
MARRIAGE_encoder = OneHotEncoder(inputCol='MARRIAGEIndex',outputCol='MARRIAGEVec')

In [0]:
input_cols_OneHot= ['LIMIT_BAL', 'SEXVec', 'EDUCATIONVec', 'MARRIAGEVec', 'AGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
assembler_OneHot = VectorAssembler(inputCols= input_cols_OneHot ,outputCol='features')

input_cols_Label= ['LIMIT_BAL', 'SEXIndex', 'EDUCATIONIndex', 'MARRIAGEIndex', 'AGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
assembler_Label = VectorAssembler(inputCols=input_cols_Label,outputCol='features')

- LogisticRegression

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier

with pipeline we define the steps that we want to do

In [0]:
from pyspark.ml import Pipeline

In [0]:
log_reg_df = LogisticRegression(featuresCol='features',labelCol='Default')
rf_df = RandomForestClassifier(featuresCol='features',labelCol='Default')

In [0]:
# a pipeline for logistic regression
pipeline_lr_OneHot = Pipeline(stages=[SEX_indexer,EDUCATION_indexer,MARRIAGE_indexer,
                            SEX_encoder,EDUCATION_encoder,MARRIAGE_encoder,
                           assembler_OneHot,log_reg_df])

pipeline_lr_Label = Pipeline(stages=[SEX_indexer,EDUCATION_indexer,MARRIAGE_indexer,
                           assembler_Label,log_reg_df])

pipeline_rf_Label = Pipeline(stages=[SEX_indexer,EDUCATION_indexer,MARRIAGE_indexer,
                           assembler_Label,rf_df])

In [0]:
# random split with a seed number to reproduce the results
train_df, test_df = df.randomSplit([0.8,.2],110)

In [0]:
fit_model_OneHot = pipeline_lr_OneHot.fit(train_df)
fit_model_Label = pipeline_lr_Label.fit(train_df)
fit_rf_Label = pipeline_rf_Label.fit(train_df)

In [0]:
train_OneHot_results = fit_model_OneHot.transform(train_df)
test_OneHot_results = fit_model_OneHot.transform(test_df)

train_Label_results = fit_model_Label.transform(train_df)
test_Label_results = fit_model_Label.transform(test_df)

- AUC
- Accuracy

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
AUC_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Default')
Accu_evaluator = MulticlassClassificationEvaluator()
Accu_evaluator.setPredictionCol('prediction')
Accu_evaluator.setLabelCol('Default')

Out[39]: MulticlassClassificationEvaluator_9c7ea24795fb

In [0]:
test_Label_results.select('Default','prediction').show(5)

+-------+----------+
Default|prediction|
+-------+----------+
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
+-------+----------+
only showing top 5 rows

In [0]:
AUC_test_Label = AUC_eval.evaluate(test_Label_results)
AUC_train_Label = AUC_eval.evaluate(train_Label_results)

AUC_test_OneHot = AUC_eval.evaluate(test_OneHot_results)
AUC_train_OneHot = AUC_eval.evaluate(train_OneHot_results)

In [0]:
Accu_test_Label = Accu_evaluator.evaluate(test_Label_results, {Accu_evaluator.metricName: "accuracy"})
Accu_train_Label = Accu_evaluator.evaluate(train_Label_results, {Accu_evaluator.metricName: "accuracy"})

Accu_test_OneHot = Accu_evaluator.evaluate(test_OneHot_results, {Accu_evaluator.metricName: "accuracy"})
Accu_train_OneHot = Accu_evaluator.evaluate(train_OneHot_results, {Accu_evaluator.metricName: "accuracy"})

In [0]:
print("Test AUC with OneHot Encoding", AUC_test_OneHot)
print("\n")
print("Train AUC with OneHot Encoding", AUC_train_OneHot)

Test AUC with OneHot Encoding 0.6743270267915413


Train AUC with OneHot Encoding 0.6720942978336184

In [0]:
print("Accuracy: Test with OneHot Encoding", Accu_test_OneHot)
print("\n")
print("Accuracy: Train with OneHot Encoding", Accu_train_OneHot)

Accuracy: Test with OneHot Encoding 0.6741658515378763


Accuracy: Train with OneHot Encoding 0.6721243808475509

In [0]:
print("Test AUC with Label Encoding", AUC_test_Label)
print("\n")
print("Train AUC with Label Encoding", AUC_train_Label)

Test AUC with Label Encoding 0.6742163899904006


Train AUC with Label Encoding 0.6720674417402163

In [0]:
print("Accuracy: Test with Label Encoding", Accu_test_Label)
print("\n")
print("Accuracy: Train with Label Encoding", Accu_train_Label)

Accuracy: Test with Label Encoding 0.6740571676991631


Accuracy: Train with Label Encoding 0.6720968629609246

Feature Selection
- More generalizable more 
- avoid over fitting
- prediction in train set and test set would be closer but it's not guaranteed

In [0]:
fit_rf_Label.stages[-1].featureImportances

Out[47]: SparseVector(23, {0: 0.0536, 1: 0.0004, 2: 0.0003, 3: 0.0004, 4: 0.0017, 5: 0.3544, 6: 0.1827, 7: 0.1175, 8: 0.0725, 9: 0.0436, 10: 0.0439, 11: 0.0179, 12: 0.0063, 13: 0.004, 14: 0.0076, 15: 0.0033, 16: 0.0025, 17: 0.0243, 18: 0.038, 19: 0.01, 20: 0.0067, 21: 0.0016, 22: 0.007})

In [0]:
input_cols_Label

Out[48]: ['LIMIT_BAL',
 'SEXIndex',
 'EDUCATIONIndex',
 'MARRIAGEIndex',
 'AGE',
 'PAY_1',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6']

In [0]:
df2=fit_rf_Label.transform(train_df)

In [0]:
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

In [0]:
#with this you can extract the features, since the outcome is a pandas dataframe, we can use pandas related functions and methods
# you can check it with this command: type()
ExtractFeatureImp(fit_rf_Label.stages[-1].featureImportances, df2, "features")

,idx,name,vals,score
2,5,PAY_1,NaN,0.354374
3,6,PAY_2,NaN,0.182674
4,7,PAY_3,NaN,0.117484
5,8,PAY_4,NaN,0.072501
0,0,LIMIT_BAL,NaN,0.053582
7,10,PAY_6,NaN,0.043897
6,9,PAY_5,NaN,0.043617
15,18,PAY_AMT2,NaN,0.038000
14,17,PAY_AMT1,NaN,0.024279
8,11,BILL_AMT1,NaN,0.017907


In [0]:
features=ExtractFeatureImp(fit_rf_Label.stages[-1].featureImportances, df2, "features")

features=features[features.score>0.00036].iloc[:,1].tolist()
vars=features+['Default']
vars

Out[52]: ['PAY_1',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'LIMIT_BAL',
 'PAY_6',
 'PAY_5',
 'PAY_AMT2',
 'PAY_AMT1',
 'BILL_AMT1',
 'PAY_AMT3',
 'BILL_AMT4',
 'PAY_AMT6',
 'PAY_AMT4',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT5',
 'BILL_AMT6',
 'AGE',
 'PAY_AMT5',
 'MARRIAGEIndex',
 'Default']

In [0]:
train_df_selected=fit_model_Label.transform(train_df).select(vars)
test_df_selected=fit_model_Label.transform(test_df).select(vars)

In [0]:
# we don't need to apply whole of pipeline just the last step which training the model is enough
assembler = VectorAssembler(inputCols= features, outputCol="features")
datatrain = assembler.transform(train_df_selected)
datatest = assembler.transform(test_df_selected)

log_reg_df = LogisticRegression(featuresCol='features',labelCol='Default')

fit_new = log_reg_df.fit(datatrain)

In [0]:
train_new_results = fit_new.transform(datatrain)
test_new_results = fit_new.transform(datatest)

In [0]:
AUC_test_new = AUC_eval.evaluate(test_new_results)
AUC_train_new = AUC_eval.evaluate(train_new_results)

Accu_test_new = Accu_evaluator.evaluate(test_new_results, {Accu_evaluator.metricName: "accuracy"})
Accu_train_new = Accu_evaluator.evaluate(train_new_results, {Accu_evaluator.metricName: "accuracy"})

In [0]:
print("Test AUC with Label Encoding & feature selection", AUC_test_new)
print("Train AUC with Label Encoding & feature selection", AUC_train_new)
print("\n")
print("Accuracy: Test with Label Encoding & feature selection", Accu_test_new)
print("Accuracy: Train with Label Encoding & feature selection", Accu_train_new)

Test AUC with Label Encoding & feature selection 0.6744415617503133
Train AUC with Label Encoding & feature selection 0.6738538331075274


Accuracy: Test with Label Encoding & feature selection 0.6742745353765895
Accuracy: Train with Label Encoding & feature selection 0.6738855255916346

- Tuning the algorithms (CrossValidator) for LogisticRegression
- As we indicate 3 values for regParam, 3 values for maxIter, and 3 values for elasticNetParam, this grid will have 3 x 3 x 3 = 27 parameter settings for CrossValidator to choose from. We will create a 5-fold cross validator.

In [0]:
import pyspark.ml.tuning as tune
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [0]:
lc_df = LogisticRegression(featuresCol='features',labelCol='Default')

In [0]:
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lc_df.regParam, [0.01, 0.5, 2.0])
             .addGrid(lc_df.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lc_df.maxIter, [1, 5, 10])
             .build())

In [0]:
cv = CrossValidator(estimator=lc_df, estimatorParamMaps=paramGrid, evaluator=BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Default'),numFolds=5)

In [0]:
cvModel = cv.fit(datatrain)

In [0]:
train_new0_results = cvModel.transform(datatrain)
test_new0_results = cvModel.transform(datatest)

AUC_test_new0 = AUC_eval.evaluate(test_new0_results)
AUC_train_new0 = AUC_eval.evaluate(train_new0_results)

Accu_test_new0 = Accu_evaluator.evaluate(test_new0_results, {Accu_evaluator.metricName: "accuracy"})
Accu_train_new0 = Accu_evaluator.evaluate(train_new0_results, {Accu_evaluator.metricName: "accuracy"})

In [0]:
print("Test AUC with Label Encoding & feature selection", AUC_test_new0)
print("Train AUC with Label Encoding & feature selection", AUC_train_new0)
print("\n")
print("Accuracy: Test with Label Encoding & feature selection", Accu_test_new0)
print("Accuracy: Train with Label Encoding & feature selection", Accu_train_new0)

Test AUC with Label Encoding & feature selection 0.6836516797232108
Train AUC with Label Encoding & feature selection 0.6814271560353627


Accuracy: Test with Label Encoding & feature selection 0.6825345071187915
Accuracy: Train with Label Encoding & feature selection 0.6816180517336269

Trying other algorithms
- Decision tree
- Random forest
- Gradient boosting tree
- SVM

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier

In [0]:
dt_df = DecisionTreeClassifier(featuresCol='features',labelCol='Default')

fit_new1 = dt_df.fit(datatrain)

In [0]:
train_new1_results = fit_new1.transform(datatrain)
test_new1_results = fit_new1.transform(datatest)

In [0]:
AUC_test_new1 = AUC_eval.evaluate(test_new1_results)
AUC_train_new1 = AUC_eval.evaluate(train_new1_results)

Accu_test_new1 = Accu_evaluator.evaluate(test_new1_results, {Accu_evaluator.metricName: "accuracy"})
Accu_train_new1 = Accu_evaluator.evaluate(train_new1_results, {Accu_evaluator.metricName: "accuracy"})

In [0]:
print("Test AUC with Label Encoding & feature selection", AUC_test_new1)
print("Train AUC with Label Encoding & feature selection", AUC_train_new1)
print("\n")
print("Accuracy: Test with Label Encoding & feature selection", Accu_test_new1)
print("Accuracy: Train with Label Encoding & feature selection", Accu_train_new1)

Test AUC with Label Encoding & feature selection 0.7091163590130414
Train AUC with Label Encoding & feature selection 0.7091098167785622


Accuracy: Test with Label Encoding & feature selection 0.7082925768938159
Accuracy: Train with Label Encoding & feature selection 0.7092460099064392

In [0]:
from pyspark.ml.classification import RandomForestClassifier

In [0]:
rf_df = RandomForestClassifier(featuresCol='features',labelCol='Default')
fit_new2 = rf_df.fit(datatrain)

In [0]:
train_new2_results = fit_new2.transform(datatrain)
test_new2_results = fit_new2.transform(datatest)

In [0]:
AUC_test_new2 = AUC_eval.evaluate(test_new2_results)
AUC_train_new2 = AUC_eval.evaluate(train_new2_results)

Accu_test_new2 = Accu_evaluator.evaluate(test_new2_results, {Accu_evaluator.metricName: "accuracy"})
Accu_train_new2 = Accu_evaluator.evaluate(train_new2_results, {Accu_evaluator.metricName: "accuracy"})

In [0]:
print("Test AUC with Label Encoding & feature selection", AUC_test_new2)
print("Train AUC with Label Encoding & feature selection", AUC_train_new2)
print("\n")
print("Accuracy: Test with Label Encoding & feature selection", Accu_test_new2)
print("Accuracy: Train with Label Encoding & feature selection", Accu_train_new2)

Test AUC with Label Encoding & feature selection 0.7177751829960568
Train AUC with Label Encoding & feature selection 0.716640825419494


Accuracy: Test with Label Encoding & feature selection 0.7170959678295837
Accuracy: Train with Label Encoding & feature selection 0.716758392955421

In [0]:
from pyspark.ml.classification import GBTClassifier

In [0]:
gbt_df = GBTClassifier(featuresCol='features',labelCol='Default')
fit_new3 = gbt_df.fit(datatrain)

In [0]:
train_new3_results = fit_new3.transform(datatrain)
test_new3_results = fit_new3.transform(datatest)

AUC_test_new3 = AUC_eval.evaluate(test_new3_results)
AUC_train_new3 = AUC_eval.evaluate(train_new3_results)

Accu_test_new3 = Accu_evaluator.evaluate(test_new3_results, {Accu_evaluator.metricName: "accuracy"})
Accu_train_new3 = Accu_evaluator.evaluate(train_new3_results, {Accu_evaluator.metricName: "accuracy"})

In [0]:
print("Test AUC with Label Encoding & feature selection", AUC_test_new3)
print("Train AUC with Label Encoding & feature selection", AUC_train_new3)
print("\n")
print("Accuracy: Test with Label Encoding & feature selection", Accu_test_new3)
print("Accuracy: Train with Label Encoding & feature selection", Accu_train_new3)

Test AUC with Label Encoding & feature selection 0.7258639309569661
Train AUC with Label Encoding & feature selection 0.7313676208635289


Accuracy: Test with Label Encoding & feature selection 0.7253559395717857
Accuracy: Train with Label Encoding & feature selection 0.731452944413869

In [0]:
from pyspark.ml.classification import LinearSVC

In [0]:
svm_df = LinearSVC(featuresCol='features',labelCol='Default')
fit_new4 = svm_df.fit(datatrain)

In [0]:
train_new4_results = fit_new4.transform(datatrain)
test_new4_results = fit_new4.transform(datatest)

AUC_test_new4 = AUC_eval.evaluate(test_new4_results)
AUC_train_new4 = AUC_eval.evaluate(train_new4_results)

Accu_test_new4 = Accu_evaluator.evaluate(test_new4_results, {Accu_evaluator.metricName: "accuracy"})
Accu_train_new4 = Accu_evaluator.evaluate(train_new4_results, {Accu_evaluator.metricName: "accuracy"})

In [0]:
print("Test AUC with Label Encoding & feature selection", AUC_test_new4)
print("Train AUC with Label Encoding & feature selection", AUC_train_new4)
print("\n")
print("Accuracy: Test with Label Encoding & feature selection", Accu_test_new4)
print("Accuracy: Train with Label Encoding & feature selection", Accu_train_new4)

Test AUC with Label Encoding & feature selection 0.6859154697983977
Train AUC with Label Encoding & feature selection 0.6840616444466532


Accuracy: Test with Label Encoding & feature selection 0.6849255515704815
Accuracy: Train with Label Encoding & feature selection 0.684232250963126

As we indicate 4 values for maxDepth and 3 values for maxBin, this grid will have 4 x 3 = 12 parameter settings for CrossValidator to choose from. We will create a 5-fold CrossValidator.

In [0]:
# Create ParamGrid for Cross Validation
paramGrid1 = (ParamGridBuilder()
             .addGrid(dt_df.maxDepth, [1, 2, 6, 10])
             .addGrid(dt_df.maxBins, [20, 40, 80])
             .build())

In [0]:
# Create 5-fold CrossValidator
cv1 = CrossValidator(estimator=dt_df, estimatorParamMaps=paramGrid1, evaluator=BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Default'),numFolds=5)

In [0]:
# Run cross validations
cvModel1 = cv1.fit(datatrain)

In [0]:
train_new01_results = cvModel1.transform(datatrain)
test_new01_results = cvModel1.transform(datatest)

AUC_test_new01 = AUC_eval.evaluate(test_new01_results)
AUC_train_new01 = AUC_eval.evaluate(train_new01_results)

Accu_test_new01 = Accu_evaluator.evaluate(test_new01_results, {Accu_evaluator.metricName: "accuracy"})
Accu_train_new01 = Accu_evaluator.evaluate(train_new01_results, {Accu_evaluator.metricName: "accuracy"})

In [0]:
print("Test AUC with Label Encoding & feature selection", AUC_test_new01)
print("Train AUC with Label Encoding & feature selection", AUC_train_new01)
print("\n")
print("Accuracy: Test with Label Encoding & feature selection", Accu_test_new01)
print("Accuracy: Train with Label Encoding & feature selection", Accu_train_new01)

Test AUC with Label Encoding & feature selection 0.7333074275164758
Train AUC with Label Encoding & feature selection 0.7645795124963434


Accuracy: Test with Label Encoding & feature selection 0.7331811759591349
Accuracy: Train with Label Encoding & feature selection 0.764611997798569

As we indicate 4 values for maxDepth, 3 values for maxBin, and 2 values for numTrees, this grid will have 4 x 3 x 2 = 24 parameter settings for CrossValidator to choose from. We will create a 5-fold CrossValidator.

In [0]:
# Create ParamGrid for Cross Validation
paramGrid2 = (ParamGridBuilder()
             .addGrid(rf_df.maxDepth, [1, 2, 6, 10])
             .addGrid(rf_df.maxBins, [20, 40, 80])
             .addGrid(rf_df.numTrees, [5, 20])
             .build())

In [0]:
# Create 5-fold CrossValidator
cv2 = CrossValidator(estimator=rf_df, estimatorParamMaps=paramGrid2, evaluator=BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Default'),numFolds=5)

In [0]:
# Run cross validations
cvModel2 = cv2.fit(datatrain)

In [0]:
train_new02_results = cvModel2.transform(datatrain)
test_new02_results = cvModel2.transform(datatest)

AUC_test_new02 = AUC_eval.evaluate(test_new02_results)
AUC_train_new02 = AUC_eval.evaluate(train_new02_results)

Accu_test_new02 = Accu_evaluator.evaluate(test_new02_results, {Accu_evaluator.metricName: "accuracy"})
Accu_train_new02 = Accu_evaluator.evaluate(train_new02_results, {Accu_evaluator.metricName: "accuracy"})

In [0]:
print("Test AUC with Label Encoding & feature selection", AUC_test_new02)
print("Train AUC with Label Encoding & feature selection", AUC_train_new02)
print("\n")
print("Accuracy: Test with Label Encoding & feature selection", Accu_test_new02)
print("Accuracy: Train with Label Encoding & feature selection", Accu_train_new02)

Test AUC with Label Encoding & feature selection 0.761141312513927
Train AUC with Label Encoding & feature selection 0.781501063980174


Accuracy: Test with Label Encoding & feature selection 0.7606781871535703
Accuracy: Train with Label Encoding & feature selection 0.7815905338470005

The best model is Random forest（tuning with the ParamGridBuilder and the CrossValidator.）
- Test AUC with Label Encoding & feature selection 0.761141312513927

- Accuracy: Test with Label Encoding & feature selection 0.7606781871535703